In [14]:
AREAS = [
"Semantic segmentation",
"Image classification",
"Object detection",
"Object Recognition",
"Domain adaptation",
"Image generation",
"Data augmentation (Image augmentation)",
"Pose estimation",
"Autonomous vehicles",
"Denoising",
"Super-Resolution",
"Object Tracking",
"Action Recognition",
"Face Recognition",
"Depth Estimation",
"Optical Character Recognition",
"3D Reconstruction",
"Image Retrieval",
"Optical Flow Estimation",
"Style Transfer",
"Image Compression"]

# Google Scholar Crawl

In [21]:
from scholarly import scholarly
from tqdm import tqdm

In [28]:
gs_results = {}
total_paper = 150
for area in tqdm(AREAS):
    search_query = scholarly.search_pubs(area)
    area_results = []
    for _ in range(total_paper):
        raw = next(search_query)
        if raw is None:
            break
        result = [raw['bib'], raw['pub_url']]
        area_results.append(result)
        

  5%|▍         | 1/21 [00:23<07:55, 23.77s/it]

In [27]:
next(search_query)

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Bisenet: Bilateral segmentation network for real-time semantic segmentation',
  'author': ['C Yu', 'J Wang', 'C Peng', 'C Gao'],
  'pub_year': '2018',
  'venue': 'Proceedings of the …',
  'abstract': 'Semantic segmentation requires both rich spatial information and sizeable receptive field. However, modern approaches usually compromise spatial resolution to achieve real-time inference speed, which leads to poor performance. In this paper, we address this dilemma'},
 'filled': False,
 'gsrank': 5,
 'pub_url': 'http://openaccess.thecvf.com/content_ECCV_2018/html/Changqian_Yu_BiSeNet_Bilateral_Segmentation_ECCV_2018_paper.html',
 'author_id': ['Hv-vj2sAAAAJ', 'GStTsxAAAAAJ', 'agmuPioAAAAJ', '4tku-lwAAAAJ'],
 'num_citations': 759,
 'url_scholarbib': '/scholar?q=info:-BVpJufY9i0J:scholar.google.com/&output=cite&scirp=4&hl=en',
 'url_add_sclib': '/cita